In [1]:
try:
    import pymongo
    from pymongo import MongoClient
except ImportError:
    raise ImportError('PyMongo is not installed')
import csv,json,wbpy,sys, getopt, pprint,requests
import pandas as pd
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import time

In [2]:
connection = pymongo.MongoClient('localhost',27017)
database= connection['DAPDatabase']
collection=database['GlobalTemperaturesCountryCollection']
data = collection.find()
connection.close()
listValuesCountryTemp = list(data)
dfCountryTemp = pd.DataFrame(listValuesCountryTemp) 
dfCountryTemp.head()
dfCountryTemp = dfCountryTemp[["dt","AverageTemperature","Country"]]
dfCountryTemp.columns = ['Date',"AverageTemperature","Country"]

In [46]:
dfCountryTemp[["AverageTemperature"]].head()
dfCountryTemp['AverageTemperature'] = dfCountryTemp['AverageTemperature'].replace("","xyz")
dfCountryTemp['AverageTemperature'] = dfCountryTemp['AverageTemperature'].replace("xyz",None)
dfCountryTemp = dfCountryTemp.dropna(subset=['AverageTemperature'])
    
global_temp_country_clear = dfCountryTemp[~dfCountryTemp['Country'].isin(['Denmark', 'Antarctica', 'France', 'Europe', 'Netherlands','United Kingdom', 'Africa', 'South America'])]
global_temp_country_clear = global_temp_country_clear.replace(['Denmark (Europe)', 'France (Europe)', 'Netherlands (Europe)', 'United Kingdom (Europe)'],['Denmark', 'France', 'Netherlands', 'United Kingdom'])
countries = np.unique(global_temp_country_clear['Country'])
mean_temp = []
    
global_temp_country_clear.dtypes
global_temp_country_clear['AverageTemperature']=global_temp_country_clear['AverageTemperature'].astype(float)

for country in countries:
    mean_temp.append(global_temp_country_clear[global_temp_country_clear['Country'] == country]['AverageTemperature'].mean())
    data = [ dict(type = 'choropleth',locations = countries,z = mean_temp,locationmode = 'country names',text = countries,marker = dict( line = dict(color = 'rgb(0,0,0)', width = 1)), colorbar = dict(autotick = True, tickprefix = '', title = '# Average\nTemperature,\n°C'))]
    layout = dict(title = 'Average land temperature in countries', geo = dict(showframe = False, showocean = True,oceancolor = 'rgb(0,255,255)',projection = dict(type = 'orthographic',rotation = dict(lon = 60, lat = 10),), lonaxis =  dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)'
        ),
    lataxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)'
            )
        ),
    )

fig = dict(data=data, layout=layout)
py.iplot(fig, validate=False, filename='worldmap')


In [7]:
    connection = pymongo.MongoClient('localhost',27017)
    database= connection['DAPDatabase']
    collection=database['GlobalTemperaturesCollection']
    data = collection.find()
    connection.close()
    listValuesGlobalTemp = list(data)
    dfGlobalTemp = pd.DataFrame(listValuesGlobalTemp) 
    dfGlobalTemp.head()

,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,_id,dt
0,,,3.0340000000000003,3.574,,,,,5dee46af2b94e04427c82b19,1750-01-01
1,,,3.083,3.702,,,,,5dee46af2b94e04427c82b1a,1750-02-01
2,,,5.626,3.076,,,,,5dee46af2b94e04427c82b1b,1750-03-01
3,,,8.49,2.451,,,,,5dee46af2b94e04427c82b1c,1750-04-01
4,,,11.573,2.072,,,,,5dee46af2b94e04427c82b1d,1750-05-01


In [8]:
    years = np.unique(dfGlobalTemp['dt'].apply(lambda x: x[:4]))
    mean_temp_world = []
    mean_temp_world_uncertainty = []
    dfGlobalTemp['LandAverageTemperature'].isnull().sum()
    dfGlobalTemp['LandAverageTemperature'] = dfGlobalTemp['LandAverageTemperature'].replace("","xyz")
    dfGlobalTemp['LandAverageTemperature'] = dfGlobalTemp['LandAverageTemperature'].replace("xyz",None)
    dfGlobalTemp['LandAverageTemperature']=dfGlobalTemp['LandAverageTemperature'].astype(float)

    dfGlobalTemp['LandAverageTemperatureUncertainty'].isnull().sum()    
    dfGlobalTemp['LandAverageTemperatureUncertainty'] = dfGlobalTemp['LandAverageTemperatureUncertainty'].replace("","xyz")
    dfGlobalTemp['LandAverageTemperatureUncertainty'] = dfGlobalTemp['LandAverageTemperatureUncertainty'].replace("xyz",None)
    dfGlobalTemp['LandAverageTemperatureUncertainty']=dfGlobalTemp['LandAverageTemperatureUncertainty'].astype(float)
    
    dfGlobalTemp = dfGlobalTemp.dropna(subset=['LandAverageTemperature'])
    dfGlobalTemp = dfGlobalTemp.dropna(subset=['LandAverageTemperatureUncertainty'])

    for year in years:
        mean_temp_world.append(dfGlobalTemp[dfGlobalTemp['dt'].apply(
                lambda x: x[:4]) == year]['LandAverageTemperature'].mean())
        mean_temp_world_uncertainty.append(dfGlobalTemp[dfGlobalTemp['dt'].apply(
                lambda x: x[:4]) == year]['LandAverageTemperatureUncertainty'].mean())
    
    trace0 = go.Scatter(
            x = years, 
            y = np.array(mean_temp_world) + np.array(mean_temp_world_uncertainty),
            fill= None,
            mode='lines',
            name='Uncertainty top',
            line=dict(
                    color='rgb(0, 255, 255)',
                    )
            )
    trace1 = go.Scatter(
            x = years, 
            y = np.array(mean_temp_world) - np.array(mean_temp_world_uncertainty),
            fill='tonexty',
            mode='lines',
            name='Uncertainty bot',
            line=dict(
                    color='rgb(0, 255, 255)',
                            )
                    )

    trace2 = go.Scatter(
            x = years, 
            y = mean_temp_world,
            name='Average Temperature',
            line=dict(
                    color='rgb(199, 121, 093)',
                    )
            )
    data = [trace0, trace1, trace2]

    layout = go.Layout(
            xaxis=dict(title='year'),
            yaxis=dict(title='Average Temperature, °C'),
            title='Average land temperature in world',
            showlegend = False)

    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)


In [10]:
    connection = pymongo.MongoClient('localhost',27017)
    database= connection['DAPDatabase']
    collection=database['seaLevelCollection']
    data = collection.find()
    connection.close()
    listValues = list(data)
    df = pd.DataFrame(listValues) 
    df.head()
    df = df[['GMSL','Time']]
    df.columns = ['GMSL','Date']
    df = df[["Date","GMSL"]] 

In [14]:
    df['GMSL'].isnull().sum()
    df['Date'].isnull().sum()
    yearsFromDate = np.unique(df['Date'].apply(lambda x: x[:4]))
    mean_sea_level= []
    for year in yearsFromDate:
        mean_sea_level.append(df[df['Date'].apply(
                lambda x: x[:4]) == year]['GMSL'].mean())
     
    
    trace = go.Scatter(
        x = yearsFromDate, 
        y = mean_sea_level,
        name='Average Sea Level',
        line=dict(
                color='rgb(199, 121, 093)',
                )
        )
    data = [trace]
    layout = go.Layout(
            xaxis=dict(title='year'),
            yaxis=dict(title='Average sea level'),
            title='Average Sea Level Rise',
            showlegend = False)

    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

In [26]:
    connection = pymongo.MongoClient('localhost',27017)
    database= connection['DAPDatabase']
    collection=database['precipitationCollection']
    data = collection.find()
    connection.close()
    listValuesAPIPrec = list(data)
    listValuesAPIPrec[0:2]
    xPrec =[]
    aPrec= []
    for i in listValuesAPIPrec:
        for k in i:
            if k != "_id":
                xPrec.append(i.get(k))
                aPrec.append(k)
    randomListPrec=[]  
    randomList1Prec=[]   
    randomList2Prec=[]    
    randomList3Prec=[]
    listx = []
    for e in aPrec:
        randomList1Prec.append(e)
        for  z in xPrec:
            if aPrec.index(e) == xPrec.index(z):
                for q in z:
                   
                # randomList.append(randomList1[-1])
                    randomList2Prec.append(q)
                    randomList2Prec.append(z.get(q))
                #randomList.append(randomList1)
                    randomListPrec.append([randomList1Prec[-1],randomList2Prec[-1],randomList2Prec[-2]])
                        #randomList.append(randomList2[-2])
 
               # randomLt.append(z.get(q))
     
    dfAPIPrec = pd.DataFrame(randomListPrec)  
    dfAPIPrec.columns = ['Country',"AveragePrecipitation","Year"]
        


In [31]:
    connection = pymongo.MongoClient('localhost',27017)
    database= connection['DAPDatabase']
    collection=database['temperatureCollection']
    data = collection.find()
    connection.close()
    listValuesAPITemp = list(data)
    listValuesAPITemp[0:2]
    x =[]
    a= []
    for i in listValuesAPITemp:
        for k in i:
            if k != "_id":
                x.append(i.get(k))
                a.append(k)
    randomList=[]  
    randomList1=[]   
    randomList2=[]    
    randomList3=[]
    listx = []
    for e in a:
        randomList1.append(e)
        for  z in x:
            if a.index(e) == x.index(z):
                for q in z:
                   
                # randomList.append(randomList1[-1])
                    randomList2.append(q)
                    randomList2.append(z.get(q))
                #randomList.append(randomList1)
                    randomList.append([randomList1[-1],randomList2[-1],randomList2[-2]])
                        #randomList.append(randomList2[-2])
              
               # randomLt.append(z.get(q))
     
    dfAPITemp = pd.DataFrame(randomList)  
    dfAPITemp.columns = ['Country',"AverageTemperature","Year"]

In [41]:
    dfAPIPrec['AveragePrecipitation'].isnull().sum()
    dfAPITemp['AverageTemperature'].isnull().sum()
    yearsFromDateTempAPI = np.unique(dfAPITemp['Year'])
    yearsFromDatePrecAPI = np.unique(dfAPIPrec['Year'])
    dfAPIPrec.dtypes
    dfAPITemp.dtypes
    DiffTempCount=[]
    DiffPrecCount=[]
    AvgTempCount=[]
    AvgPrecCount=[]
    countriesUnique = np.unique(dfAPITemp['Country'])
  
    for country in countriesUnique:
        DiffTempCount.append(dfAPITemp[dfAPITemp['Country'] == country]['AverageTemperature'].max()-dfAPITemp[dfAPITemp['Country'] == country]['AverageTemperature'].min())
        DiffPrecCount.append(dfAPIPrec[dfAPIPrec['Country'] == country]['AveragePrecipitation'].max()-dfAPIPrec[dfAPIPrec['Country'] == country]['AveragePrecipitation'].min())
        AvgTempCount.append(dfAPITemp[dfAPITemp['Country'] == country]['AverageTemperature'].mean())
        AvgPrecCount.append(dfAPIPrec[dfAPIPrec['Country'] == country]['AveragePrecipitation'].mean())
        
        
        
    trace0 = go.Scatter(
        x = yearsFromDateTempAPI, 
        y = dfAPITemp[dfAPITemp['Country']=='Australia']['AverageTemperature'],
        mode='lines',
        name='Australia',
        line=dict(
                color='rgb(199, 121, 093)',
                )
        )
        
    trace1 = go.Scatter(
        x = yearsFromDateTempAPI, 
        y = dfAPITemp[dfAPITemp['Country']=='India']['AverageTemperature'],
        mode='lines',
        name='India',
        line=dict(
                color='rgb(0, 255, 255)',
                        )
                )
    trace2 = go.Scatter(
        x = yearsFromDateTempAPI, 
        y = dfAPITemp[dfAPITemp['Country']=='Ireland']['AverageTemperature'],
        mode='lines',
        name='Ireland',
        line=dict(
                color='rgb(35, 190, 148)',
                        )
                )
    trace3 = go.Scatter(
        x = yearsFromDateTempAPI, 
        y = dfAPITemp[dfAPITemp['Country']=='United States of America']['AverageTemperature'],
        mode='lines',
        name='United States of America',
        line=dict(
                color='rgb(140, 80, 190)',
                        )
                )
    trace4 = go.Scatter(
        x = yearsFromDateTempAPI, 
        y = dfAPITemp[dfAPITemp['Country']=='Greece']['AverageTemperature'],
        mode='lines',
        name='Greece',
        line=dict(
                color='rgb(220, 130, 50)',
                        )
                )
    
        
    data = [trace0,trace1,trace2,trace3,trace4]
    layout = go.Layout(
            xaxis=dict(title='year'),
            yaxis=dict(title='Average Temperature'),
            title='Average temperature of random 5 countries in world'
            )

    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

In [45]:
    dfAPIPrec['AveragePrecipitation'].isnull().sum()
    dfAPITemp['AverageTemperature'].isnull().sum()
    yearsFromDateTempAPI = np.unique(dfAPITemp['Year'])
    yearsFromDatePrecAPI = np.unique(dfAPIPrec['Year'])
    dfAPIPrec.dtypes
    dfAPITemp.dtypes
    DiffTempCount=[]
    DiffPrecCount=[]
    AvgTempCount=[]
    AvgPrecCount=[]
    countriesUnique = np.unique(dfAPITemp['Country'])
  
    for country in countriesUnique:
        DiffTempCount.append(dfAPITemp[dfAPITemp['Country'] == country]['AverageTemperature'].max()-dfAPITemp[dfAPITemp['Country'] == country]['AverageTemperature'].min())
        DiffPrecCount.append(dfAPIPrec[dfAPIPrec['Country'] == country]['AveragePrecipitation'].max()-dfAPIPrec[dfAPIPrec['Country'] == country]['AveragePrecipitation'].min())
        AvgTempCount.append(dfAPITemp[dfAPITemp['Country'] == country]['AverageTemperature'].mean())
        AvgPrecCount.append(dfAPIPrec[dfAPIPrec['Country'] == country]['AveragePrecipitation'].mean())
        
        
        
    trace0 = go.Scatter(
        x = yearsFromDatePrecAPI, 
        y = dfAPIPrec[dfAPIPrec['Country']=='Australia']['AveragePrecipitation'],
        mode='lines',
        name='Australia',
        line=dict(
                color='rgb(199, 121, 093)',
                )
        )
        
    trace1 = go.Scatter(
        x = yearsFromDatePrecAPI, 
        y = dfAPIPrec[dfAPIPrec['Country']=='India']['AveragePrecipitation'],
        mode='lines',
        name='India',
        line=dict(
                color='rgb(220, 130, 50)',
                        )
                )
    trace2 = go.Scatter(
        x = yearsFromDatePrecAPI, 
        y = dfAPIPrec[dfAPIPrec['Country']=='Ireland']['AveragePrecipitation'],
        mode='lines',
        name='Ireland',
        line=dict(
                color='rgb(35, 190, 50)',
                        )
                )
    trace3 = go.Scatter(
        x = yearsFromDatePrecAPI, 
        y = dfAPIPrec[dfAPIPrec['Country']=='United States of America']['AveragePrecipitation'],
        mode='lines',
        name='United States of America',
        line=dict(
                color='rgb(140, 80, 120)',
                        )
                )
    trace4 = go.Scatter(
        x = yearsFromDatePrecAPI, 
        y = dfAPIPrec[dfAPIPrec['Country']=='Greece']['AveragePrecipitation'],
        mode='lines',
        name='Greece',
        line=dict(
                color='rgb(0, 255, 255)',
                        )
                )
    
        
    data = [trace0,trace1,trace2,trace3,trace4]
    layout = go.Layout(
            xaxis=dict(title='year'),
            yaxis=dict(title='Average Precipitation'),
            title='Average precipitation of random 5 countries in world'
            )

    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)